In [83]:
import geopy
import folium
import pandas as pd
import time
import numpy as np

In [3]:
#replace the api_key with your own, can be registered for free at mapquest.com
geolocator = geopy.geocoders.OpenMapQuest(api_key="niWwLqudLfy4UvuvoXACLzV1eA8wGCGx")

In [6]:
#file should be stored in xlsx (Excel data)
df = pd.read_excel(r'test_data.xlsx')
df.head()

,first_name,last_name,Address_Line_1,Addr-2,City,Zip
0,Ayman,Khafagi,913 Fairway Park Dr,NaN,Ann Arbor,48103-8965
1,Howard,Lerner,"555 E. William St, #20L",NaN,Ann Arbor,48104-2427
2,Eric,Ferguson,"3217 Homestead Commons Dr, Apt 7",NaN,Ann Arbor,48108-3206
3,Joshua,Richter,290 Nichols dr.,NaN,saline,48176-1019
4,Dawid,Kierys,1215 Hill St,NaN,Ann Arbor,48104-3127


In [110]:
#iterate through the members list, appending two new columns for the 
# latitude and longitude of each member
addresses = df['Address_Line_1']
zipcodes = df['Zip']
city = df['City']
seriesLength = len(df['Zip'])
latitudeSer = pd.Series(np.zeros(seriesLength),index=df.index)
longitudeSer = pd.Series(np.zeros(seriesLength),index=df.index)
#next series is to report whether the address look up failed
goodOrBad = pd.Series(np.zeros(seriesLength,dtype=bool),index=df.index)

In [111]:
#Do not run this each time! Only run it upon receiving a new data set
# Otherwise, use just the map plotting tool

#probably going to need to do a considerable amount of data clean up
# 1. get rid of apartment numbers
# get rid of drive, street, et cetera
for j, address in enumerate(addresses):
    #do some text processing first
    #only take string to comma, avoiding apartment #s
    if addresses[j].find(',') != -1:
        addressNow = addresses[j][1:addresses[j].find(',')]
    else:
        addressNow = addresses[j]
    addressNow = addressNow.replace(".", '')
    
    #get the location
    time.sleep(0.1)
    location = geolocator.geocode(addressNow + ' ' + zipcodes[j])
    if location == None:
        #try adding the city
        newAddress = addressNow + ' ' + city[j] + ' ' + zipcodes[j]
        location = geolocator.geocode(newAddress)
        if location == None:
            #FAILED, set to center of Ann Arbor
            goodOrBad[j] = 0;
            location = geolocator.geocode('Ann Arbor, MI')
        else:
            goodOrBad[j] = 1;
            locationCoords = (location.latitude, location.longitude)
    else:
        goodOrBad[j] = 1;
        locationCoords = (location.latitude, location.longitude)
    print(addressNow + ' ' + zipcodes[j] + str(locationCoords))
    latitudeSer[j] = location.latitude
    longitudeSer[j] = location.longitude
df['latitude'] = latitudeSer
df['longitude'] = longitudeSer
df['goodOrBad'] = goodOrBad

913 Fairway Park Dr 48103-8965(42.272143, -83.8350055)
55 E William St 48104-2427(42.2778784, -83.7463256)
217 Homestead Commons Dr 48108-3206(42.24216, -83.716136)
290 Nichols dr 48176-1019(42.24216, -83.716136)
1215 Hill St 48104-3127(42.2718547, -83.7417714)


In [133]:
df.head()
#export the data to a .xlsx file
df.to_excel('MembersListWithCoords.xlsx')

In [134]:
#define the map here, add the latitude and longitude points, with names and addresses
#only plot out the points that are TRUE in goodOrBad column
AnnArborCoords = geolocator.geocode('Ann Arbor, MI')
m = folium.Map(location=[AnnArborCoords.latitude, AnnArborCoords.longitude])

validation = df['goodOrBad']
#add a circle for each point in the members list
for j, address in enumerate(df['Address_Line_1']):
      if validation[j] == True:
            #could be modified to include phone number
            toPrint = df['first_name'][j] + ' ' + df['last_name'][j] 
            folium.CircleMarker(
            popup=toPrint,
            radius=3,
            location=(df['latitude'][j], df['longitude'][j]),
            color = '#ff0000',
            fill=True,
            fill_color = "#ff0000#"
            ).add_to(m)
m